In [1]:
from collections import Counter
import itertools

import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import linalg 
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from itertools import zip_longest

df = pd.read_csv('../input/abcnews-date-text.csv')
nltk.download('stopwords')
stopwords_set = set(stopwords.words('english'))
df['tokenized'] = df.headline_text.str.split(' ')

df['length'] = df.tokenized.map(len)
df = df.loc[df.length > 1]


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#explode and sanitize

ex = df.explode('tokenized')

ex = ex.loc[ex.tokenized.str.len() > 2]

ex = ex.loc[~ex.tokenized.isin(stopwords_set)]

In [31]:
#ceate dictionary of words

dictionary = ex.tokenized.drop_duplicates()

dictionary = pd.Series(dictionary.tolist(), name='words')

dictionary = dictionary.to_frame()

dictionary_lookup = dictionary.to_dict()['words']

dictionary['encode'] = dictionary.index

dictionary = dictionary.set_index('words')

dictionary.encode = dictionary.encode
dictionary = dictionary.to_dict()['encode']


{'aba': 0,
 'decides': 1,
 'community': 2,
 'broadcasting': 3,
 'licence': 4,
 'act': 5,
 'fire': 6,
 'witnesses': 7,
 'must': 8,
 'aware': 9,
 'defamation': 10,
 'calls': 11,
 'infrastructure': 12,
 'protection': 13,
 'summit': 14,
 'air': 15,
 'staff': 16,
 'aust': 17,
 'strike': 18,
 'pay': 19,
 'rise': 20,
 'affect': 21,
 'australian': 22,
 'travellers': 23,
 'ambitious': 24,
 'olsson': 25,
 'wins': 26,
 'triple': 27,
 'jump': 28,
 'antic': 29,
 'delighted': 30,
 'record': 31,
 'breaking': 32,
 'barca': 33,
 'aussie': 34,
 'qualifier': 35,
 'stosur': 36,
 'wastes': 37,
 'four': 38,
 'memphis': 39,
 'match': 40,
 'addresses': 41,
 'security': 42,
 'council': 43,
 'iraq': 44,
 'australia': 45,
 'locked': 46,
 'war': 47,
 'timetable': 48,
 'opp': 49,
 'contribute': 50,
 'million': 51,
 'aid': 52,
 'take': 53,
 'robson': 54,
 'celebrates': 55,
 'birthday': 56,
 'bathhouse': 57,
 'plans': 58,
 'move': 59,
 'ahead': 60,
 'big': 61,
 'hopes': 62,
 'launceston': 63,
 'cycling': 64,
 'champ

In [32]:

encode = ex.tokenized.map(dictionary.get).to_frame()

encode

,tokenized
0,0
0,1
0,2
0,3
0,4
...,...
1244183,1319
1244183,112468
1244183,6182
1244183,450


In [33]:

encode.index.astype('int')
encode.tokenized.astype('int')
docs = encode.groupby(encode.index)['tokenized'].apply(list)



In [34]:

encoded_docs = list(zip(*zip_longest(*docs.to_list(), fillvalue=0)))

encoded_docs = np.asarray(encoded_docs)



In [48]:

from scipy.sparse import dok_matrix


from scipy.sparse import csr_array

term_document_matrix  = dok_matrix((len(dictionary),np.size(encoded_docs, 0)), dtype=int)
for encoded_term in dictionary.values():
    row_column_index = np.argwhere(encoded_docs==encoded_term).T
    term_map = csr_array((np.ones(row_column_index[0].size), (row_column_index[0], row_column_index[1])), shape=encoded_docs.shape, dtype=int)
    term_document_matrix[encoded_term] = term_map.sum( axis=1).T

term_document_matrix.toarray()

array([1, 0, 0, ..., 0, 0, 0])